
# Using *sedonalib* to model a supernova spectrum 

In this example we will use the *sedonalib* python package to create, run, and view a model of a supernova spectrum

To add python tools to your python path, for now you can set

```export PYTHONPATH=$PYTHONPATH:$SEDONA_HOME/tools/sedonalib```

where $SEDONA_HOME points to the base directory of your sedona installation

## Creating an input model file using sedonalib  

The first step is create a model file that describes the initial conditions (i.e., density, velocity, composition) of our supernova ejecta.

*Sedonalib* has built in functions for easily creating a spherically symmetric model where the velocity is homologous (i.e., velocity = radius/time) as is often the case for supernovae.  We simply choose the mass and kinetic energy of the ejecta and the class sets the proper velocity structure and power-law density profile.


In [ ]:
import sedonalib as sed
import matplotlib.pyplot as plt


# model parameters
mass = 1.99e33
KE   = 1e51
day  = 60.0*60.0*24.0   # seconds in a day

# Define a new 1D spherical model with 100 zones
mod = sed.new_model("1D_sphere",dims=100)

# Set a homologous broken-power law density structure 
# (other opttions are "exponential" or "constant")
mod.set_homologous_profile(mass,KE,"powerlaw",time=day)

# Set the initial temperature to be everywhere = 10^4 K
# (the code will solve for the real T structure)
mod.set_constant_temperature(10000.0)

# set the composition to be a 50/50 mix 
# of 14Si and 56Ni everywhere
mod.add_elements(["14.28","28.56"])
mod.set_constant_composition([0.5,0.5])

# Write to a file
mod.write("testmodel.mod")
print(mod)


## Creating a parameter file with sedonalib

We next  create a parameter file that describes all of the runtime parameters for our calculation. Sedona parameter files are relatively simple text files (using the lua scripting language) that can easily be constructed by hand, but *sedonalib* provides a handy python class for generating and modifying them.

The *sedonalib* parameter file class comes with some templates that prefills the values of the runtime parameters with values appropriate for a certain kind of science run.  Here we will start with the template for a Type Ia supernova spectrum.

If you do not want to use a template, you can just create an empty parameter class using

```param = sed.new_paramfile()```

and then change the values of runtime parameters as shown in the box below


In [ ]:
# print out the available templates
sed.print_parameter_templates()

# create a parm object based on the typeIa supernova spectrum one 
param = sed.new_paramfile('supernova/1D_typeIa_supernova_spectrum')

# print out the parameter file values
print (param)

We now can change some of the parameters in our param file to chose the type of run we want.

Here we will change the name of the model file to the name of the model we created above, the time since explosion that the spectrum is to be run at, and the number of photon packets to use.

We then write out the parameter file to a file named "myparams.lua"

In [ ]:
days = 3600.0*24.0

param["model_file"]                   = "testmodel.mod"
param["tstep_time_start"]             = 25*days
param["particles_n_emit_radioactive"] = 1e4

# write the parameter file to disk with a file name "myparams.lua"
param.write("myparams.lua")

## Run the code

We copy over the executable *sedona6.ex* from the Sedona src/ directory and run it passing the myparams.lua parameter file we created above.  

This assumes that the code has been compiled (such that *sedona6.ex* is in the src/ directory) and that you have set the $SEDONA_HOME environment variable.

In [ ]:
# copy over the executable in the src/directory
import os
executable = os.getenv('SEDONA_HOME') + '/src/sedona6.ex'
os.system("cp " + executable + " ./")

# run the code
!./sedona6.ex myparams.lua

# to run on multiple processors with mpi you would use instead
# mpirun -np 2 ./sedona6.ex myparams.lua 
# where -np sets the number of ranks to run with 

## Read and plot up results

If the code executed properly it should have made 4 spectrum files, one for each iteration. Hopefully by the last (4th iteration) the temperature structure of the model has converged. 

We can take a look at the output of the last iteration by using *sedonalib* to read the "spectrum_4.h5" file into a spectrum class and then asking for the wavelength and luminosity arrays.

Sedona calculates a specific luminosity, $L_\lambda$, (units ergs/sec/angstrom) of the model. To convert to the flux detected on earth, you would divide $L_\lambda$ by $4 \pi D^2$ where D is the distance of the supernova from earth.

In [ ]:
import matplotlib.pyplot as plt

spec = sed.read("spectrum_4.h5",spec_units='angstroms')
lam,L_lam = spec.get_spectrum()


plt.plot(lam,L_lam)
plt.xlabel("wavelength (angstroms)")
plt.ylabel("specific luminosity (ergs/s/angstrom)")

plt.show()